In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

#Explicit waits
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

In [3]:
# Infinite scrolling tweet scraping
website = "https://twitter.com/search?q=python&src=typed_query"
s = Service("C:\\bin\\chromedriver.exe")
driver = webdriver.Chrome(service=s)
driver.get(website)
driver.maximize_window


# Extracts the tweet text and the user name 
def get_tweet(ele):
    try:
        user = ele.find_element(By.XPATH, ".//span[contains(text(), '@')]").text
        text = ele.find_element(By.XPATH, ".//div[@lang]").text
        tweet_data = [user, text]
    except:
        tweet_data = ["user", "data"]
    return tweet_data

user_data = []
tweet_data = []
tweet_ids = set()

scrolling = True
while scrolling == True:
    # Makes sure the page is loaded before continuing
    tweets = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located(
        (By.XPATH, "//article[@role='article']")
    ))
    
    # Get each tweet element from the loaded page
    # It is a assumed an average of 15 tweets are loaded per page
    for tweet in tweets[-15:]:
        tweet_list = get_tweet(tweet)
        tweet_id = "".join(tweet_list)    # Makes sure it is a unique tweet
        
        if tweet_id not in tweet_ids:
            tweet_ids.add(tweet_id)
            user_data.append(tweet_list[0])
            tweet_data.append(" ".join(tweet_list[1].split()))
    
    # Scrolls the page for new tweets
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # Reach till the end of the scrool
        # if new_height == last_height:
        #     scrolling = False
        #     break
        
        # For scrapping a set number of tweets 
        if len(user_data) > 100:
            scrolling = False
            break
        else: 
            last_height = new_height
            break

driver.quit()

In [7]:
df_tweets = pd.DataFrame({"user": user_data, "text": tweet_data})
df_tweets.to_csv("Twitter_inf_scroll.csv", index=False)
print(df_tweets)

                 user                                               text
0         @driscollis                      How to print emoji in #Python
1         @AlSweigart  I created a short and simple Python module to ...
2      @Antipolluters  More than 20 Trusts have spent £46,000 on the ...
3    @DataScientistsN  Don’t let ASUU strike stop your learning. This...
4       @philipvollet  Edit your DataFrame like a spreadsheet! Mito i...
..                ...                                                ...
120    @anthonypjshaw  Benchmarked 2.5mil executions and the direct_i...
121          @attomos  day 42 of #100DaysOfCode Add unit tests to my ...
122  @PavaniMidathana  #Day2 of #1hourofcode Today i completed lists ...
123  @zhangyi14377506  Finish python program with this size keyboard ...
124       @lai_yannie  Figuring out how to clean data? And maybe just...

[125 rows x 2 columns]
